# 📊 03 - Comparison of Bayesian LSTM, Standard LSTM, and Prophet


In this notebook, we compare the performance of:
- Bayesian LSTM (with Monte Carlo Dropout)
- Standard LSTM (no dropout at test time)
- Prophet (additive model from Facebook)

The comparison includes:
- Forecast accuracy (RMSE, MAE)
- Confidence intervals (where available)
- Inference speed


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
!pip install git+https://gitlab2.tamucc.edu/cpati/bayesian-lstm.git
from bayeslstm import BayesianLSTM, vectorized_mc_dropout_predict

  Cloning https://gitlab2.tamucc.edu/cpati/bayesian-lstm.git to c:\users\hp\appdata\local\temp\pip-req-build-e6b31cl3
  Resolved https://gitlab2.tamucc.edu/cpati/bayesian-lstm.git to commit 5f1c8825007f150e46bc0095b42cab1fec125c2a
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://gitlab2.tamucc.edu/cpati/bayesian-lstm.git 'C:\Users\Hp\AppData\Local\Temp\pip-req-build-e6b31cl3'

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Load Melbourne temperature data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
df = pd.read_csv(url, parse_dates=['Date'])
df.columns = ['ds', 'y']
scaler = MinMaxScaler()
df['scaled'] = scaler.fit_transform(df[['y']])

# Create sequences for LSTM
def create_sequences(data, seq_len=30):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

X, y = create_sequences(df['scaled'].values)
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

train_X, test_X = X_tensor[:3000], X_tensor[3000:]
train_y, test_y = y_tensor[:3000], y_tensor[3000:]


In [7]:
class StandardLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StandardLSTM, self).__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

std_model = StandardLSTM(1, 64, 1)
optimizer = torch.optim.Adam(std_model.parameters(), lr=0.005)
loss_fn = torch.nn.MSELoss()

std_model.train()
for epoch in range(20):
    idx = torch.randperm(train_X.shape[0])
    for i in range(0, len(train_X), 64):
        xb = train_X[idx[i:i+64]]
        yb = train_y[idx[i:i+64]]
        optimizer.zero_grad()
        pred = std_model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()


In [8]:
bayes_model = BayesianLSTM(1, 64, 1, dropout_rate=0.3)
optimizer = torch.optim.Adam(bayes_model.parameters(), lr=0.005)
bayes_model.train()
for epoch in range(20):
    idx = torch.randperm(train_X.shape[0])
    for i in range(0, len(train_X), 64):
        xb = train_X[idx[i:i+64]]
        yb = train_y[idx[i:i+64]]
        optimizer.zero_grad()
        pred = bayes_model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()


In [10]:
!pip install prophet

   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   --------- ------------------------------ 3.1/13.3 MB 22.3 MB/s eta 0:00:01
   -------------------------------- ------- 10.7/13.3 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 29.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/932.3 kB ? eta -:--:--
   --------------------------------------- 932.3/932.3 kB 35.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# Prepare data for Prophet
from prophet import Prophet
prophet_df = df[['ds', 'y']].iloc[:(3000+len(test_X))]
train_prophet = prophet_df.iloc[:3000]

model_prophet = Prophet()
model_prophet.fit(train_prophet)

future = model_prophet.make_future_dataframe(periods=len(test_X))
forecast = model_prophet.predict(future)
yhat_prophet = forecast['yhat'].iloc[3000:].values


C:\Users\Hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
13:13:59 - cmdstanpy - INFO - Chain [1] start processing
13:14:00 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
# Standard LSTM predictions
std_model.eval()
with torch.no_grad():
    std_preds = std_model(test_X).numpy()
std_preds_inv = scaler.inverse_transform(std_preds)
test_y_inv = scaler.inverse_transform(test_y.numpy())

# Bayesian LSTM predictions
bayes_model.eval()
mean_preds, _ = vectorized_mc_dropout_predict(bayes_model, test_X, T=100)
mean_preds_inv = scaler.inverse_transform(mean_preds)

# Prophet
prophet_inv = yhat_prophet.reshape(-1, 1)

# Metrics
def evaluate(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{model_name} → RMSE: {rmse:.4f}, MAE: {mae:.4f}")

evaluate(test_y_inv, std_preds_inv, "Standard LSTM")
evaluate(test_y_inv, mean_preds_inv, "Bayesian LSTM")
evaluate(test_y_inv, prophet_inv, "Prophet")


Standard LSTM → RMSE: 2.2119, MAE: 1.7341
Bayesian LSTM → RMSE: 2.2601, MAE: 1.7731
Prophet → RMSE: 3.2295, MAE: 2.5185
